In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
from scipy.stats import shapiro
from scipy.stats import levene
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LinearRegression

treino = pd.read_csv('train.csv') 

treino.describe()

In [ ]:
treino.info()

In [ ]:
treino.head(10)

In [ ]:
num_cols = treino.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = treino.select_dtypes(include=[object]).columns.tolist()

missing_values = treino.isnull().sum()
missing_percentage = (missing_values / len(treino)) * 100

print("Variáveis Numéricas:\n")
for col in num_cols:
    print(f"  - {col}")

print("\nVariáveis Categóricas:\n")
for col in cat_cols:
    print(f"  - {col}")

print("\nValores Faltantes:\n")
for col, val in missing_values[missing_values > 0].items():
    print(f"  - {col}: {val} valores faltantes ({missing_percentage[col]:.2f}%)")

In [ ]:
# Lista de colunas categóricas que precisam ser codificadas
categorical_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 
                    'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 
                    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                    'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                    'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 
                    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                    'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
                    'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
                    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
                    'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 
                    'SaleType', 'SaleCondition']

for column in categorical_cols:
    print(column)
    print(treino[column].unique())

In [ ]:
le = OrdinalEncoder()
for column in categorical_cols:
    treino[f"{column}_le"] = le.fit_transform(treino[column].values.reshape(-1, 1))

for column in categorical_cols:
    print(column)
    print(treino[column].unique())
    print(treino[f"{column}_le"].unique())
    print()

In [ ]:
num_cols = treino.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = treino.select_dtypes(include=[object]).columns.tolist()

missing_values = treino.isnull().sum()
missing_percentage = (missing_values / len(treino)) * 100

print("Variáveis Numéricas:\n")
for col in num_cols:
    print(f"  - {col}")

print("\nVariáveis Categóricas:\n")
for col in cat_cols:
    print(f"  - {col}")

print("\nValores Faltantes:\n")
for col, val in missing_values[missing_values > 0].items():
    print(f"  - {col}: {val} valores faltantes ({missing_percentage[col]:.2f}%)")

In [ ]:
# Normalização de variáveis numéricas
scaler = StandardScaler()
treino[num_cols] = scaler.fit_transform(treino[num_cols])

In [ ]:
# Preenchendo dados ausentes na coluna LotFrontage com a mediana
treino['LotFrontage'].fillna(treino['LotFrontage'].median(), inplace=True)
# Preenchendo dados ausentes na coluna Alley com "None"
treino['Alley'].fillna('None', inplace=True)
# Preenchendo dados ausentes na coluna PoolQC com "None"
treino['PoolQC'].fillna('None', inplace=True)
# Preenchendo dados ausentes na coluna Fence com "None"
treino['Fence'].fillna('None', inplace=True)
# Preenchendo dados ausentes na coluna MiscFeature com "None"
treino['MiscFeature'].fillna('None', inplace=True)

In [ ]:
# Selecionando features e target
features = treino.columns.tolist()
target = 'SalePrice'

X = treino[features]
y = treino[target]

# Codificando variáveis categóricas
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Separando variáveis numéricas e categóricas
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

In [ ]:
print(treino.describe())

num_cols = treino.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = treino.select_dtypes(include=[object]).columns.tolist()

missing_values = treino.isnull().sum()
missing_percentage = (missing_values / len(treino)) * 100

In [ ]:
# Criação de nova feature (ÁREA TOTAL DA CASA)
treino['TotalSF'] = treino['TotalBsmtSF'] + treino['1stFlrSF'] + treino['2ndFlrSF']

In [ ]:
plt.figure(figsize=(20, 30))
for i, col in enumerate(num_cols):
    plt.subplot(10, 4, i + 1)
    sns.histplot(treino[col], kde=True, bins=30, color='blue')
    plt.title(col)
    plt.tight_layout()
plt.show()

summary_numeric = {}
for col in num_cols:
    summary_numeric[col] = treino[col].describe()

for col, stats in summary_numeric.items():
    print(f"\nEstatísticas para coluna '{col}':\n{stats}")

In [ ]:
num_cat_cols = len(cat_cols)
num_rows = (num_cat_cols // 2) + (num_cat_cols % 2)
plt.figure(figsize=(20, num_rows * 6))

for i, col in enumerate(cat_cols, 1):
    plt.subplot(num_rows, 2, i)
    sns.histplot(x=col, data=treino, color='blue')
    plt.title(f'Distribuição de {col}')
    plt.xticks(rotation=45)
    plt.tight_layout()

plt.show()

summary_categorical = {}
for col in cat_cols:
    summary_categorical[col] = treino[col].describe()

for col, stats in summary_categorical.items():
    print(f"\nEstatísticas para coluna '{col}':\n{stats}")

In [ ]:
correlations = treino[num_cols].corr()['SalePrice'].sort_values(ascending=False)

top_correlations = correlations.head(11)
print("Top 10 correlações com SalePrice:\n", top_correlations)

bottom_correlations = correlations.tail(10)
print("\nBottom 10 correlações com SalePrice:\n", bottom_correlations)

In [ ]:
anova_results = []

for col in cat_cols:
    categories = treino[col].unique()
    
    if all(treino[treino[col] == category]['SalePrice'].count() > 1 for category in categories):
        try:
            anova = f_oneway(*[treino[treino[col] == category]['SalePrice'] for category in categories])
            anova_results.append((col, anova.statistic, anova.pvalue))
        except Exception as e:
            print(f"Erro ao calcular ANOVA para {col}: {e}")

df_anova = pd.DataFrame(anova_results, columns=['Variável Categórica', 'F-statistic', 'p-value'])

df_anova.sort_values(by='p-value', ascending=True, inplace=True)

top_10_correlations = df_anova.head(10)

print("As 10 maiores correlações:")
print(top_10_correlations)

bottom_10_correlations = df_anova.tail(10)

print("\nAs 10 menores correlações:")
print(bottom_10_correlations)

In [ ]:
plt.figure(figsize=(40, 20))
sns.heatmap(treino[num_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
s_palette = 'viridis'

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='Neighborhood', y='SalePrice', data=treino, palette='coolwarm', errorbar=None)
plt.title('SalePrice por Neighbourhood')
plt.xlabel('Neighbourhood')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='OverallQual', y='SalePrice', data=treino, palette='YlOrBr', errorbar=None)
plt.title('SalePrice por OverallQual')
plt.xlabel('OverallQual')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='GrLivArea', y='SalePrice', data=treino, alpha=0.7 , hue='SalePrice', palette=s_palette)
plt.title('SalePrice por GrLivArea')
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.legend().remove()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='GarageCars', y='SalePrice', data=treino, palette='Blues', errorbar=None)
plt.title('SalePrice por GarageCars')
plt.xlabel('GarageCars')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='GarageArea', y='SalePrice', data=treino, color='blue')
plt.title('SalePrice por GarageArea (Line Plot)')
plt.xlabel('GarageArea')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='TotalBsmtSF', y='SalePrice', data=treino, color='blue')
plt.title('SalePrice por TotalBsmtBF')
plt.xlabel('TotalBsmtBF')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='1stFlrSF', y='SalePrice', data=treino, alpha=0.7 , hue='SalePrice', palette=s_palette)
plt.title('SalePrice por 1stFlrSF')
plt.xlabel('1stFlrSF')
plt.ylabel('SalePrice')
plt.legend().remove()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='FullBath', y='SalePrice', data=treino, palette='magma', errorbar=None)
plt.title('SalePrice por FullBath')
plt.xlabel('FullBath')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='TotRmsAbvGrd', y='SalePrice', data=treino, palette='Spectral', errorbar=None)
plt.title('SalePrice por TotRmsAbvGrd')
plt.xlabel('TotRmsAbvGrd')
plt.ylabel('SalePrice')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='YearBuilt', y='SalePrice', data=treino, alpha=0.7, hue='SalePrice', palette=s_palette)
plt.title('SalePrice por YearBuilt')
plt.xlabel('YearBuilt')
plt.ylabel('SalePrice')
plt.legend().remove()  
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='YearRemodAdd', y='SalePrice', data=treino, alpha=0.7 , hue='SalePrice', palette=s_palette)
plt.title('SalePrice por YearRemodAdd')
plt.xlabel('YearRemodAdd')
plt.ylabel('SalePrice')
plt.legend().remove()
plt.show()

In [ ]:
top_cat_cols = [
    'ExterQual', 'KitchenQual', 'Foundation', 
    'SaleCondition', 'SaleType', 'MSZoning', 'HouseStyle', 
    'LotShape', 'CentralAir'
]

filtered_data = treino[top_cat_cols + ['SalePrice']]

plt.figure(figsize=(18, 12))
for i, feature in enumerate(top_cat_cols, 1):
    plt.subplot(3, 4, i)
    sns.barplot(x=feature, y='SalePrice', data=filtered_data, palette='mako', errorbar=None)
    plt.title(f'SalePrice por {feature}')
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
neighborhood_prices = treino.groupby('Neighborhood')['SalePrice'].mean().sort_values(ascending=False)

top_neighborhoods = neighborhood_prices.head(5).index.tolist()
print("Bairros com maior preço médio:")
print(top_neighborhoods)

In [ ]:
filtered_data = treino[treino['Neighborhood'].isin(top_neighborhoods)]

variables_of_interest = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
    '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd' ,
    'ExterQual', 'KitchenQual', 'Foundation', 'HeatingQC', 'SaleCondition'
]

num_plots = len(variables_of_interest)
rows = (num_plots // 4) + 1

plt.figure(figsize=(18, 12))
for i, feature in enumerate(variables_of_interest, 1):
    plt.subplot(rows, 4, i)
    if treino[feature].dtype == 'object':  
        sns.histplot(filtered_data[feature], bins=20, kde=True, stat='density')
        plt.xlabel(feature)
        plt.ylabel('Contagem')
    else:
        sns.histplot(filtered_data[feature], bins=20, kde=True, stat='density')
        plt.xlabel(feature)
        plt.ylabel('Densidade')
    plt.title(f'{feature} - Bairros com Maior Preço Médio')
    
plt.tight_layout()
plt.show()

In [ ]:
neighborhood_prices = treino.groupby('Neighborhood')['SalePrice'].mean().sort_values(ascending=True)

bottom_neighborhoods = neighborhood_prices.head(5).index.tolist()
print("Bairros com menor preço médio:")
print(bottom_neighborhoods)

In [ ]:
filtered_data = treino[treino['Neighborhood'].isin(bottom_neighborhoods)]

variables_of_interest = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
    '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd',
    'ExterQual', 'KitchenQual', 'Foundation', 'HeatingQC', 'SaleCondition'
]

num_plots = len(variables_of_interest)
rows = (num_plots // 4) + 1

plt.figure(figsize=(18, 12))
for i, feature in enumerate(variables_of_interest, 1):
    plt.subplot(rows, 4, i)
    if treino[feature].dtype == 'object':  
        sns.histplot(filtered_data[feature], bins=20, kde=True, stat='density')
        plt.xlabel(feature)
        plt.ylabel('Média de SalePrice')
    else:
        sns.histplot(filtered_data[feature], bins=20, kde=True, stat='density')
        plt.xlabel(feature)
        plt.ylabel('Densidade')
    plt.title(f'{feature} - Bairros com Menor Preço Médio')
    
plt.tight_layout()
plt.show()

In [ ]:
shapiro_results = {}
for col in num_cols:
    data = treino[col].dropna() 
    stat, p_value = shapiro(data)
    shapiro_results[col] = {'Estatística': stat, 'Valor p': p_value}

for col, result in shapiro_results.items():
    print(f"\nTeste de Shapiro-Wilk para '{col}':")
    print(f"  Estatística: {result['Estatística']}")
    print(f"  Valor p: {result['Valor p']}")

In [ ]:
levene_results = {}

for col in num_cols:
    stat, p_value = levene(treino[col].dropna(), treino['SalePrice'].dropna())
    levene_results[col] = {'Estatística': stat, 'Valor p': p_value}

for col, result in levene_results.items():
    print(f"\nTeste de Levene para '{col}':")
    print(f"  Estatística: {result['Estatística']}")
    print(f"  Valor p: {result['Valor p']}")

In [ ]:
null_indicatores_num = treino[num_cols].isnull().astype(int)
null_indicatores_num['SalePrice'] = treino['SalePrice']

null_correlacao = null_indicatores_num.corr()['SalePrice'].sort_values(ascending=False)

maiores_corr_numer = null_correlacao.head(10) 
menores_corr_numer = null_correlacao.tail(10)

print("Top 10 maiores correlações entre valores nulos e SalePrice:\n", maiores_corr_numer)
print("\nTop 10 menores correlações entre valores nulos e SalePrice:\n", menores_corr_numer)

In [ ]:
plt.figure(figsize=(10, 6))
maiores_corr_numer[1:].plot(kind='bar', color='green')
plt.title('Top 10 maiores correlações entre valores nulos (numéricas) e SalePrice')
plt.ylabel('Correlação')
plt.xlabel('Colunas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
null_indicatores_cat = treino[cat_cols].isnull().astype(int)
null_indicatores_cat['SalePrice'] = treino['SalePrice']

null_correlacao_cat = null_indicatores_cat.corr()['SalePrice'].sort_values(ascending=False)

maiores_corr_cat = null_correlacao_cat.head(20)
menores_corr_cat = null_correlacao_cat.tail(10)

print("Maiores correlações entre valores nulos (categóricas) e SalePrice:\n", maiores_corr_cat)
print("\nMenores correlações entre valores nulos (categóricas) e SalePrice:\n", menores_corr_cat)

In [ ]:
plt.figure(figsize=(10, 6))
maiores_corr_cat[1:].plot(kind='bar', color='blue')
plt.title('Top 10 maiores correlações entre valores nulos (categóricas) e SalePrice')
plt.ylabel('Correlação')
plt.xlabel('Colunas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Variáveis numéricas
numeric_cols_with_missing = treino[num_cols].columns[treino[num_cols].isnull().any()]
for col in numeric_cols_with_missing:
    treino[col].fillna(treino[col].median(), inplace=True)

# Variáveis categóricas
categorical_cols_with_missing = treino[cat_cols].columns[treino[cat_cols].isnull().any()]
for col in categorical_cols_with_missing:
    treino[col].fillna(treino[col].mode()[0], inplace=True)

In [ ]:
num_cols = [
    'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 
    'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
    'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
    'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 
    'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 
    'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 
    'MoSold', 'YrSold'
]

categorical_features = [
    'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 
    'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
    'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
    'MasVnrType', 'Foundation', 'BsmtQual', 'BsmtCond', 
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 
    'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 
    'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 
    'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 
    'MiscFeature', 'SaleType', 'SaleCondition'
]

le = OrdinalEncoder()
for column in categorical_features:
    treino[f"{column}_le"] = le.fit_transform(treino[column].values.reshape(-1, 1))

In [ ]:
scaler = StandardScaler()
treino[num_cols] = scaler.fit_transform(treino[num_cols])

In [ ]:
# Seleção de features numéricas com alta correlação
num_features = [
    'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 
    'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 
    'YearBuilt', 'YearRemodAdd'
]

# Seleção de features categóricas com alta correlação
cat_features = [
    'Neighborhood', 'ExterQual', 'KitchenQual', 'Foundation', 
    'SaleCondition', 'SaleType', 'MSZoning', 'HouseStyle', 
    'LotShape', 'CentralAir'
]

# Combinar features numéricas e categóricas
selected_features = num_features + cat_features
X = treino[selected_features]
y = treino['SalePrice']

# Pré-processamento para dados numéricos: imputação de valores ausentes e normalização
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pré-processamento para dados categóricos: imputação de valores ausentes e codificação one-hot
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Combinar etapas de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

# Criar um pipeline que inclui pré-processamento e o modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [ ]:
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Treinar o modelo
pipeline.fit(X_train, y_train)

In [ ]:
# Avaliar o modelo
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

In [ ]:
# Cross-validation score
cv_score = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_score)
print("Cross-validated RMSE:", cv_rmse.mean())

In [ ]:
# Selecionar as features
num_features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', '1stFlrSF']
cat_features = ['Neighborhood', 'ExterQual', 'KitchenQual', 'Foundation', 'SaleCondition']

X = treino[num_features + cat_features]
y = treino['SalePrice']

# Pré-processamento para dados numéricos: imputação de valores ausentes e normalização
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pré-processamento para dados categóricos: imputação de valores ausentes e codificação one-hot
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar etapas de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

# Criar um pipeline que inclui pré-processamento e o modelo de regressão linear Multipla
pipeline_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
pipeline_reg.fit(X_train, y_train)

# Fazer previsões
y_pred_reg = pipeline_reg.predict(X_test)

# Avaliar o desempenho do modelo
mse_reg = mean_squared_error(y_test, y_pred_reg)
rmse_reg = np.sqrt(mse_reg)
print(f"Mean Squared Error (Regressão Linear): {mse_reg:.4f}")
print(f"Root Mean Squared Error (Regressão Linear): {rmse_reg:.4f}")

In [ ]:
# Definindo um limiar para determinar se o preço de venda é alto ou baixo
threshold = np.percentile(treino['SalePrice'], 75)  # preço acima do 75º percentil é considerado alto

# Criando a variável binária de saída
treino['SalePrice_bin'] = treino['SalePrice'].apply(lambda x: 1 if x >= threshold else 0)

# Verificando o balanceamento das classes
print(treino['SalePrice_bin'].value_counts())

In [ ]:
# Selecionar as mesmas features usadas na regressão linear
X = treino[num_features + cat_features]
y_bin = treino['SalePrice_bin']

# Combinar etapas de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

# Criar um pipeline que inclui pré-processamento e o modelo de classificação (Logistic Regression)
pipeline_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2, random_state=42)

# Treinar o modelo
pipeline_clf.fit(X_train, y_train)

In [ ]:
# Fazer previsões
y_pred_clf = pipeline_clf.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred_clf)
print(f"Acurácia do modelo de classificação: {accuracy:.2f}")

# Mostrar relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_clf))